# Big Data Examination
# Roll No. - DS5B-2121
## Question 2

Q.II Considering the Item_Outlet_Sales as dependent variable in “Big Mart Sales” dataset, determine the accuracy of the model. Split the dataset according to your last digit of roll no. (Example: if your roll no is ending with 0, the ratio will be 70, 30; if your roll no is ending with 1, the ratio will be 71, 29; if your roll no is ending with 2, the ratio will be 72, 28; if your roll no is ending with 3, the ratio will be 73, 27 etc.).

### Importing Pyspark Library

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 69.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=bf2c426d86882346b14a6fc9d4f4fe8f8f6c664e20345f63052db0e57c752d41
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


## Import Library and Creating Session

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
session = SparkSession.builder.appName("Big_Mart_Sales_Dataset").master("local").getOrCreate()
#we reassign value of __name__ (inbuilt variable) to "__main__" and main is used as entry point in many languages like C++ and Java,
# else the value of name might be different

## Reading the Data From CSV

In [ ]:
data = session.read.csv("Big Mart Sale.csv", header = True, inferSchema=True)

To print top 10 raw in dataset

In [ ]:
data.show(10)

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Superma

## Check Null Values in columns

In [ ]:
from pyspark.sql.functions import isnan, when, count, col

In [ ]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+---------------+-----------+----------------+---------------+---------+--------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+---------+--------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+
|              0|       1463|               0|              0|        0|       0|                0|                        0|       2410|                   0|          0|                0|
+---------------+-----------+----------------+---------------+---------+--------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+



## Importing sql functions

In [ ]:
import pyspark.sql.functions as func 

In [ ]:
data.agg(func.percentile_approx("Item_Weight", 0.5).alias("mean")).show()

+----+
|mean|
+----+
|12.6|
+----+



## Fill Null Values 

In [ ]:
# replace 12.6 in place of Null values in Item_weight column because it is mean in this column
data = data.na.fill(value=12.6,subset=["Item_Weight"])

In [ ]:
# return Medium in place of Null values in Outlet_Size Column Because Medium is the median in Outlet_Size Column
data = data.na.fill(value="Medium",subset=["Outlet_Size"])

In [ ]:
data.show()

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|
|          DRC01|       5.92|         Regular|    0.019278216|         Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|Superma

## Exploration Of Dataset

To print all columns name

In [ ]:
data.columns

['Item_Identifier',
 'Item_Weight',
 'Item_Fat_Content',
 'Item_Visibility',
 'Item_Type',
 'Item_MRP',
 'Outlet_Identifier',
 'Outlet_Establishment_Year',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type',
 'Item_Outlet_Sales']

To count total numbers of raws in dataset

In [ ]:
data.count()

8523

In [ ]:
data.printSchema()

root
 |-- Item_Identifier: string (nullable = true)
 |-- Item_Weight: double (nullable = false)
 |-- Item_Fat_Content: string (nullable = true)
 |-- Item_Visibility: double (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Item_MRP: double (nullable = true)
 |-- Outlet_Identifier: string (nullable = true)
 |-- Outlet_Establishment_Year: integer (nullable = true)
 |-- Outlet_Size: string (nullable = false)
 |-- Outlet_Location_Type: string (nullable = true)
 |-- Outlet_Type: string (nullable = true)
 |-- Item_Outlet_Sales: double (nullable = true)



To know data type of each columns

In [ ]:
data.dtypes

[('Item_Identifier', 'string'),
 ('Item_Weight', 'double'),
 ('Item_Fat_Content', 'string'),
 ('Item_Visibility', 'double'),
 ('Item_Type', 'string'),
 ('Item_MRP', 'double'),
 ('Outlet_Identifier', 'string'),
 ('Outlet_Establishment_Year', 'int'),
 ('Outlet_Size', 'string'),
 ('Outlet_Location_Type', 'string'),
 ('Outlet_Type', 'string'),
 ('Item_Outlet_Sales', 'double')]

## Data Preprocessing
Here we convert the data into machine readable form

In [ ]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
# It is use for mapping a string columm to a index column that will be treated as a categorical column by spark.
# It is feature transformer that combine multiple columns into a single vector column.
# Pyspark ml models takes only one independent variable and one dependent varibale
#but, we have multiple independent variabales, so we use vector assembler to convert them into a single list
# of independent variables

In [ ]:
# Doing String Indexing
str_index = StringIndexer(inputCols = ['Item_Identifier','Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type'], outputCols = ['Item_Identifier1','Item_Fat_Content1','Item_Type1','Outlet_Identifier1','Outlet_Establishment_Year1','Outlet_Size1','Outlet_Location_Type1','Outlet_Type1'], handleInvalid = "skip", stringOrderType= "frequencyDesc")

In [ ]:
# One Hot Encoding
one_hot = OneHotEncoder(inputCols =['Item_Identifier1','Item_Fat_Content1','Item_Type1','Outlet_Identifier1','Outlet_Establishment_Year1','Outlet_Size1','Outlet_Location_Type1','Outlet_Type1'],  outputCols=['Item_Identifier2','Item_Fat_Content2','Item_Type2','Outlet_Identifier2','Outlet_Establishment_Year2','Outlet_Size2','Outlet_Location_Type2','Outlet_Type2'] )

In [ ]:
# APply Vector Assembler
vector_ass = VectorAssembler(inputCols = ['Item_Weight','Item_Fat_Content2','Item_Visibility','Item_Type2','Item_MRP','Outlet_Size2','Outlet_Location_Type2','Outlet_Type2'], outputCol = "allfeatures")

## Import Linear Regression and Create Model

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
linear = LinearRegression(featuresCol="allfeatures", labelCol="Item_Outlet_Sales")

# Create Pipeline for ML Model




In [ ]:
from pyspark.ml import Pipeline
mypipeline = Pipeline(stages = [str_index, one_hot, vector_ass, linear])

# Making Train Test Split

## Splitting the Dataset
# As my roll no is DS5B-2121 I will be using split as 0.71 and 0.29

In [ ]:
training, test = data.randomSplit([0.71, 0.29])

## Model Training

In [ ]:
lin_reg_model = mypipeline.fit(training)

## Test Model

In [ ]:
result = lin_reg_model.transform(test)

In [ ]:
result.show()

+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+----------------+-----------------+----------+------------------+--------------------------+------------+---------------------+------------+-------------------+-----------------+--------------+------------------+--------------------------+-------------+---------------------+-------------+--------------------+------------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|  Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|Item_Identifier1|Item_Fat_Content1|Item_Type1|Outlet_Identifier1|Outlet_Establishment_Year1|Outlet_Size1|Outlet_Location_Type1|Outlet_Type1|   Item_Identifier2|Item_Fat_Content2|    Item_Type2|Outlet_Identifier2|Outlet_Establishment_Year2| Outlet_Size2|Outlet_Location_Type2| Ou

## Evaluate Model

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
errors = ["r2", "rmse", "mse", "mae"]
name = ["R-Square or Accuracy", "Root Mean Square Error", "Mean Square Error", "Mean Absolute Error"]

for i in range(len(errors)):
  eval = RegressionEvaluator(predictionCol="prediction", labelCol='Item_Outlet_Sales', metricName=errors[i])
  print("The {} of Model is {}".format(name[i],eval.evaluate(result)))

The R-Square or Accuracy of Model is 0.5544914690571232
The Root Mean Square Error of Model is 1159.6628884131828
The Mean Square Error of Model is 1344818.0147628062
The Mean Absolute Error of Model is 853.5910257087111
